<a href="https://colab.research.google.com/github/vishwanath103/MIT-DL-Course/blob/main/Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

import tensorflow as tf

import numpy as np
import base64, io, time, gym
import IPython, functools
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

!pip install mitdeeplearning
import mitdeeplearning as mdl

     |████████████████████████████████| 2.1MB 7.6MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.2.0-cp37-none-any.whl size=2115442 sha256=8dfe124a401cdac8200499188d3e5e20b755001b067adb8d302a00ca8dfaf5a5
  Stored in directory: /root/.cache/pip/wheels/af/dc/2a/5c3633135e7e4ef4fd31463cfa1942cb1bae7486ab94e7a2ad
Successfully built mitdeeplearning


# Part 1: Cartpole

## Define the Cartpole environment and agent

### Environment

In [2]:
env = gym.make("CartPole-v0")
env.seed(1)

[1]

Our observations in this Cartpole environment are:
1. Cart position
2. Cart Velocity
3. Pole angle
4. Pole rotation rate

In [4]:
n_observations = env.observation_space
print("Environment has observation space = ", n_observations)

Environment has observation space =  Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


In [5]:
n_actions = env.action_space.n
print("Number of possible actions that the agent can choose from =", n_actions)

Number of possible actions that the agent can choose from = 2


### Cartpole agent

In [6]:
### Define the Cartpole agent ###

# Defines a feed-forward neural network
def create_cartpole_model():
  model = tf.keras.models.Sequential([
        # First Dense layer
        tf.keras.layers.Dense(units=32, activation='relu'),
        # Define the last Dense layer, which will provide the network's output
        tf.keras.layers.Dense(units=n_actions, activation=None)
  ])
  return model

cartpole_model = create_cartpole_model()

In [7]:
### Define the agents action function ###

# Fucntion that takes observations as input, executes a forward pass through model,
#   and outputs a sampled action
# Arguments:
#   model: the network that defines our agent
#   observation: observation(s) which is/are fed as input to the model
#   single: flag as to whether we are handling a single observation or batch of
#     observation, provided as an np.array
# Returns:
#   action: choice of agent action
def choose_action(model, observation, single=True):
  # add batch dimension to the observation if only a single example was provided
  observation = np.expand_dims(observation, axis=0) if single else observation

  # Feed the observations through the model to predict the log probabilities of each posible action
  logits = model.predict(observation)

  # Choose an action
  action = tf.random.categorical(logits, num_samples=1)
  action = action.numpy().flatten()
  return action[0] if single else action

## Define the agents memory

In [8]:
### Agent Memory ###

class Memory:
  def __init__(self):
    self.clear()

  # Resets/restarts the memory buffer
  def clear(self):
    self.observation = []
    self.actions = []
    self.rewards = []

  # Add observations, actions, rewards to memory
  def add_to_memory(self, new_observation, new_action, new_reward):
    self.observations.append(new_observation)
    self.actions.append(new_action)
    self.rewards.append(new_reward)

# Helper function to combine a list of Memory objects into a single Memory.
def aggregate_memories(memories):
  batch_memory = Memory()

  for memory in memories:
    for step in zip(memory.observations, memory.actions, memory.rewards):
      batch_memory.add_to_memory(*step)

  return batch_memory

# Instantiate a single Memory buffer
memory = Memory()

## Reward Function

In [9]:
### Reward function ###

# Helper function that normalizes an np.array x
def normalize(x):
  x -= np.mean(x)
  x /= np.std(x)
  return x.astype(np.float32)

# Compute normalized, discounted, cumulative rewards
# Arguments:
#   rewards: reward at timesteps in episode
#   gamma: discounting factor
# Returns:
#   normalized discounted reward
def discount_rewards(rewards, gamma=0.95):
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  for t in reversed(range(0, len(rewards))):
    # update the total discounted reward
    R = R * gamma + rewards[t]
    discounted_rewards[t] = R

  return normalize(discounted_rewards)

## Learning Algorithm